In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import os
import random

In [2]:
counts = pd.read_csv("all_samples_gene_counts.txt", sep="\t", header=0)
print(counts.columns)

Index(['Gene', 'HUVEC_RNAd0_W1_R1', 'HUVEC_RNAd0_W0_R1', 'HUVEC_RNAd0_W2_R1',
       'HUVEC_RNAd0_W3_R1', 'HUVEC_RNAd0_W1_R2', 'HUVEC_RNAd0_W2_R2',
       'HUVEC_RNAd0_W3_R2', 'HUVEC_RNAdD_W2_R2', 'HUVEC_RNAdD_W2_R1',
       'HUVEC_RNAdE_W1_R2', 'HUVEC_RNAdE_W1_R1', 'HUVEC_RNAdD_W3_R2',
       'HUVEC_RNAdD_W3_R1', 'HUVEC_RNAdE_W2_R2', 'HUVEC_RNAdE_W2_R1',
       'HUVEC_RNAdD_W1_R2', 'HUVEC_RNAdD_W1_R1', 'HUVEC_RNAdE_W3_R2',
       'HUVEC_RNAdE_W3_R1', 'HUVEC_RNAdB_W2_R1', 'HUVEC_RNAdB_W2_R2',
       'HUVEC_RNAdA_W1_R2', 'HUVEC_RNAdA_W1_R1', 'HUVEC_RNAdC_W1_R1',
       'HUVEC_RNAdC_W1_R2', 'HUVEC_RNAdB_W3_R1', 'HUVEC_RNAdB_W3_R2',
       'HUVEC_RNAdC_W2_R1', 'HUVEC_RNAdC_W2_R2', 'HUVEC_RNAdA_W2_R2',
       'HUVEC_RNAdA_W2_R1', 'HUVEC_RNAdB_W1_R1', 'HUVEC_RNAdB_W1_R2',
       'HUVEC_RNAdC_W3_R1', 'HUVEC_RNAdC_W3_R2', 'HUVEC_RNAdA_W3_R2',
       'HUVEC_RNAdA_W3_R1'],
      dtype='object')


In [3]:
metadata = pd.read_csv("metadata_rna.csv", header=0)
print(metadata)

   SampleName ExposureRate_mGh  Week  Replicate  TotalExposure_mG
0    dA_W1_R1            0.001     1          1             0.168
1    dA_W1_R2            0.001     1          2             0.168
2    dA_W2_R1            0.001     2          1             0.336
3    dA_W2_R2            0.001     2          2             0.336
4    dA_W3_R1            0.001     3          1             0.504
5    dA_W3_R2            0.001     3          2             0.504
6    dB_W1_R1             0.01     1          1             1.680
7    dB_W1_R2             0.01     1          2             1.680
8    dB_W2_R1             0.01     2          1             3.360
9    dB_W2_R2             0.01     2          2             3.360
10   dB_W3_R1             0.01     3          1             5.040
11   dB_W3_R2             0.01     3          2             5.040
12   dC_W1_R1              0.1     1          1            16.800
13   dC_W1_R2              0.1     1          2            16.800
14   dC_W2

In [4]:
count_data_only = counts.drop(columns='Gene').T
drop_inds = count_data_only != 0
count_data_only = count_data_only.loc[:,(drop_inds).any(axis=0)]
print(count_data_only.shape)


(37, 29628)


In [5]:
genes = counts['Gene'].loc[(drop_inds).any(axis=0)]
dose = [f"{metadata.loc[metadata['SampleName']==name[9:]]['TotalExposure_mG'].values[0]}Gy" for name in counts.columns[1:]]
dose_rate = [metadata.loc[metadata['SampleName']==name[9:]]['ExposureRate_mGh'].values[0] for name in counts.columns[1:]]
print(set(dose))
print(set(dose_rate))
count_data_only.columns=genes
count_data_only['Dose'] = dose
count_data_only['Dose Rate'] = dose_rate

{'33.6Gy', '50.4Gy', '0.336Gy', '1008.0Gy', '3.36Gy', '168.0Gy', '504.0Gy', '0.168Gy', '5.04Gy', '1.68Gy', '336.0Gy', '0.504Gy', '672.0Gy', '16.8Gy', '0.0Gy'}
{'0.01', 'Control', '0.1', '2.0', '1.0', '0.001'}


In [6]:
count_data_only.to_csv("data_w_rate.csv", header=True, index=False)

In [10]:
# Analyze output graphs
def analyze_graph(graph_file):
    # Create an undirected graph
    G = nx.read_gexf(graph_file)
    # Get the list of proteins
    proteins = list(G.nodes())

    # Select 1000 random pairs of proteins
    pairs = []
    for _ in range(1000):
        start_protein, end_protein = random.sample(proteins, 2)
        pairs.append((start_protein, end_protein))

    # Lists to keep track of path lengths
    path_lengths = []

    # For each pair, check if they are connected and compute average edge weight if they are
    for start_protein, end_protein in pairs:
        # Check if there is a path between the proteins
        if nx.has_path(G, start_protein, end_protein):
            try:
                path = nx.shortest_path(G, source=start_protein, target=end_protein, weight=None)
                # Compute the mean weight of the edges along the path
                # weights = []
                # for i in range(len(path) - 1):
                #     u = path[i]
                #     v = path[i + 1]
                #     edge_data = G.get_edge_data(u, v)
                #     weights.append(edge_data['weight'])
                # mean_weight = sum(weights) / len(weights)
                # Record the path length
                path_length = len(path) - 1  # Number of edges in the path
                path_lengths.append(path_length)
                # Map protein IDs to names for output
                # path_names = [protein_id_to_name.get(protein_id, protein_id) for protein_id in path]
                # start_name = protein_id_to_name.get(start_protein, start_protein)
                # end_name = protein_id_to_name.get(end_protein, end_protein)
                # Print the results with mean weight formatted to one decimal place
                print(f'Path found between {start_protein} and {end_protein}:')
                print('Path:', ' -> '.join(path))
                print(f'Path length: {path_length}')
                # print(f'Mean weight of edges: {mean_weight:.1f}\n')
            except nx.NetworkXNoPath:
                # This shouldn't happen because we already checked has_path
                print(f'No path found between {start_protein} and {end_protein}.')
        else:
            # Map protein IDs to names for output
            # start_name = protein_id_to_name.get(start_protein, start_protein)
            # end_name = protein_id_to_name.get(end_protein, end_protein)
            #print(f'No path between {start_protein} and {end_protein}.')
            continue

    # After processing all pairs, output min, max, and average path length
    if path_lengths:
        min_length = min(path_lengths)
        max_length = max(path_lengths)
        avg_length = sum(path_lengths) / len(path_lengths)
        print('--- Path Length Statistics ---')
        print(f'Minimum path length: {min_length}')
        print(f'Maximum path length: {max_length}')
        print(f'Average path length: {avg_length:.2f}')
    else:
        print('No paths found between any pairs.')

    # Compute the degrees of the nodes
    degrees = [degree for node, degree in G.degree()]
    if degrees:
        min_degree = min(degrees)
        max_degree = max(degrees)
        avg_degree = sum(degrees) / len(degrees)
        print('--- Node Degree Statistics ---')
        print(f'Minimum node degree: {min_degree}')
        print(f'Maximum node degree: {max_degree}')
        print(f'Average node degree: {avg_degree:.2f}')
    else:
        print('No nodes in the graph.')

    # Output node IDs of nodes with degree greater than 100
    high_degree_nodes = [node for node, degree in G.degree() if degree > 1000]
    if high_degree_nodes:
        print('--- Nodes with Degree Greater Than 100 ---')
        for node in high_degree_nodes:
            node_degree = G.degree(node)
            print(f'Node: {node}, Degree: {node_degree}')
    else:
        print('No nodes have a degree greater than 100.') 

In [11]:
for file in os.listdir("./"):
    if file.endswith(".gexf"): 
        print(file)
        analyze_graph(file)

dag_modularity_B_0.0Gy.gexf
Path found between TIAM2 and TRAPPC11:
Path: TIAM2 -> RALA -> PLCL1 -> YIF1B -> RAB1B -> LMAN2L -> TRAPPC3 -> TRAPPC11
Path length: 7
Path found between CRTC1 and BCL7B:
Path: CRTC1 -> CREB1 -> NCOR1 -> NFKB1 -> SMARCD3 -> BCL7B
Path length: 5
Path found between UQCRQ and NDUFAF4:
Path: UQCRQ -> NDUFA7 -> NDUFAF4
Path length: 2
Path found between ENO1 and PLBD2:
Path: ENO1 -> NME1 -> SORD -> TKTL1 -> PLBD2
Path length: 4
--- Path Length Statistics ---
Minimum path length: 2
Maximum path length: 7
Average path length: 4.50
--- Node Degree Statistics ---
Minimum node degree: 0
Maximum node degree: 100
Average node degree: 3.41
No nodes have a degree greater than 100.
dag_modularity_B_0.168Gy.gexf
Path found between CFL1 and FUT10:
Path: CFL1 -> ADK -> NIT2 -> FUT10
Path length: 3
Path found between LRRC57 and PPP2R5A:
Path: LRRC57 -> TTF1 -> LRRC8C -> PPP3CC -> PPP1R3B -> PPP2R5A
Path length: 5
--- Path Length Statistics ---
Minimum path length: 3
Maximum path